In [1]:
# Import packages and custom functions
%run 'create_final_csv_functions.py'

In [ ]:
months = ["Jan", "Feb", "March", "April","May", "June", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]   
for m in months: 
    create_final_csv_by_month(m)

In [ ]:
# Concatenate all months in one csv
concatenate_noise_sub_csv()

In [2]:
# To read from s3 bucket run
df = pd.read_csv("s3://mda.project.monaco/project_data.csv")